## 韌體燒錄

In [3]:
%esptool --port COM4 erase

Executing:
  esptool --port COM4 erase_flash

esptool.py v2.6
Serial port COM4
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
MAC: a0:20:a6:13:47:26
Uploading stub...
Running stub...
Stub running...
Erasing flash (this may take a while)...
Chip erase completed successfully in 9.4s
Hard resetting via RTS pin...


In [4]:
#%esptool --port com4 --baud 460800 write_flash --flash_size=detect -fm dio 0 ../scripts/esp8266-20190125-v1.10.bin
%esptool --port com4 esp8266 ../scripts/esp8266-20190125-v1.10.bin

Executing:
  esptool --port com4 --baud 460800 write_flash --flash_size=detect -fm dio 0 ../scripts/esp8266-20190125-v1.10.bin

esptool.py v2.6
Serial port com4
Connecting....
[Press the PRG button now if required]
Detecting chip type... ESP8266
Chip is ESP8266EX
Features: WiFi
MAC: a0:20:a6:13:47:26
Uploading stub...
Running stub...
Stub running...
Changing baud rate to 460800
Changed.
Configuring flash size...
Auto-detected Flash size: 4MB
Flash params set to 0x0240
Compressed 615388 bytes to 399928...
Wrote 615388 bytes (399928 compressed) at 0x00000000 in 9.2 seconds (effective 538.0 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


* 請先重開 Nodemcu
* ****************
## 增加檔案

In [5]:
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
Ready.


In [6]:
import os
print(os.listdir())

['boot.py']


* 修改無線網路參數

In [8]:
%sendtofile ap.txt
informatics
0953313123

Sent 2 lines (22 bytes) to ap.txt.


## 重新改寫： 
* boot.py
* main.py
* webtool.html
* webrepl_cfg.py (網路連接密碼，default：1234)

In [9]:
%sendtofile boot.py
# This file is executed on every boot (including wake-boot from deepsleep)
#import esp
#esp.osdebug(None)
import uos, machine
#uos.dupterm(None, 1) # disable REPL on UART(0)
import gc
gc.collect()
import webrepl
webrepl.start()



Sent 10 lines (229 bytes) to boot.py.


In [10]:
%sendtofile main.py
# -*- coding: utf-8 -*-

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X-%02X' %(uid[0], uid[1], uid[2], uid[3])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'upy_%s_%s' %(my_sn, sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04


# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(5, machine.Pin.OUT)


# th_sensor
th_sensor = dht.DHT11(Pin(14))


#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))

    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    th_sensor_read = request.find('GET /?th_sensor=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
   

    my_t = None
    my_h = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()    
        my_t = th_sensor.temperature()  
        my_h = th_sensor.humidity()
        print('T=', th_sensor.temperature())
        print('H=', th_sensor.humidity())

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?', 'T={}{}C'.format(my_t,'\u00b0'))
        else:
            html = html.replace('T=?', '')
        if my_h:
            html = html.replace('H=?', 'H={}{}'.format(my_h,'%'))
        else:
            html = html.replace('H=?', '')
        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()

Sent 138 lines (3221 bytes) to main.py.


In [11]:
%sendtofile webtool.html
<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?unit
</form>

<hr>
GPIO Info. :<br>
<pre>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</pre>

</html>

Sent 39 lines (789 bytes) to webtool.html.


In [12]:
%sendtofile webrepl_cfg.py
PASS = '1234'

Sent 1 lines (13 bytes) to webrepl_cfg.py.


## 取得 IP

In [16]:
import network
import time

In [17]:
sta = network.WLAN(network.STA_IF)
sta.active(True)

In [18]:
SSID='informatics'
KEY='0953313123'
sta.active(True)
print(sta.active())
sta.connect(SSID,KEY)

True


In [19]:
print(sta.isconnected())
for i in range(20):
    time.sleep(0.5)
    if sta.isconnected():
        break
print(sta.ifconfig())

True
('192.168.43.121', '255.255.255.0', '192.168.43.254', '120.118.253.1')


In [20]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.10-8-g8b7039d7d on 2019-01-26; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reWebREPL daemon started on ws://192.168.4.1:8266\r\nWebREPL daemon started on ws://192.168.43.121:8266\r\nStarted webrepl in normal mode\r\n'
not command mode

[\r\x01] b"ap config here!\r\n('ap_config:', 'informatics\\n0953313123')\r\nline to:  ('informati"cs', '0953313123')
connected!  -->  ('192.168.43.121', '255.255.255.0', '192.168.43.254', '120.118.253.1')
#8 ets_task(4020f508, 29, 3fff8fc8, 10)
[Timed out waiting for recognizable response]


## 在瀏覽器上輸入 nodemcu 所取得之 IP